In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import glob
import os
import re
from pynwb import NWBFile, TimeSeries, NWBHDF5IO
from IPython.display import display, HTML


from aind_dynamic_foraging_data_utils import nwb_utils as nu
import aind_dynamic_foraging_basic_analysis.licks.annotation as a

from aind_dynamic_foraging_basic_analysis import plot_foraging_session, compute_foraging_efficiency

In [2]:
# Load single NWB

nwb = nu.load_nwb_from_filename('/root/capsule/data/foraging_nwb_bonsai/687582_2023-11-16_12-13-24.nwb')

/opt/conda/lib/python3.9/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.5.1 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/opt/conda/lib/python3.9/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.4.0 because version 2.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/opt/conda/lib/python3.9/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.2.0 because version 0.5.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


In [3]:
nwb

,start_time,stop_time,animal_response,rewarded_historyL,rewarded_historyR,delay_start_time,goCue_start_time,reward_outcome_time,bait_left,bait_right,base_reward_probability_sum,reward_probabilityL,reward_probabilityR,reward_random_number_left,reward_random_number_right,left_valve_open_time,right_valve_open_time,block_beta,block_min,block_max,min_reward_each_block,delay_beta,delay_min,delay_max,delay_duration,ITI_beta,ITI_min,ITI_max,ITI_duration,response_duration,reward_consumption_duration,auto_waterL,auto_waterR,laser_on_trial,laser_wavelength,laser_location,laser_1_power,laser_2_power,laser_on_probability,laser_duration,laser_condition,laser_condition_probability,laser_start,laser_start_offset,laser_end,laser_end_offset,laser_protocol,laser_frequency,laser_rampingdown,laser_pulse_duration,session_wide_control,fraction_of_session,session_start_with,session_alternation,auto_train_engaged,auto_train_curriculum_name,auto_train_curriculum_version,auto_train_curriculum_schema_version,auto_train_stage,auto_train_stage_overridden,lickspout_position_x,lickspout_position_y,lickspout_position_z,reward_size_left,reward_size_right
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,4.489577e+06,4.489585e+06,2.0,False,False,4.489578e+06,4.489579e+06,4.489582e+06,False,False,0.8,0.711111,0.088889,0.963379,0.207240,0.042,0.045,20.0,20.0,40.0,1.0,0.5,1.0,1.5,1.004837,2.0,1.0,7.0,1.092118,3.0,3.0,0,0,0,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,4.0,4.0
1,4.489585e+06,4.489594e+06,2.0,False,False,4.489587e+06,4.489588e+06,4.489591e+06,True,False,0.8,0.711111,0.088889,0.526161,0.362175,0.042,0.045,20.0,20.0,40.0,1.0,0.5,1.0,1.5,1.500000,2.0,1.0,7.0,1.675733,3.0,3.0,0,0,0,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,4.0,4.0
2,4.489594e+06,4.489605e+06,2.0,False,False,4.489597e+06,4.489599e+06,4.489602e+06,True,False,0.8,0.711111,0.088889,0.904736,0.315749,0.042,0.045,20.0,20.0,40.0,1.0,0.5,1.0,1.5,1.500000,2.0,1.0,7.0,3.099859,3.0,3.0,0,0,0,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,4.0,4.0
3,4.489605e+06,4.489616e+06,2.0,False,False,4.489609e+06,4.489610e+06,4.489613e+06,True,False,0.8,0.711111,0.088889,0.758493,0.959065,0.042,0.045,20.0,20.0,40.0,1.0,0.5,1.0,1.5,1.024207,2.0,1.0,7.0,3.777559,3.0,3.0,0,0,0,NaN,None,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,4.0,4.0
,start_time,stop_time,animal_response,rewarded_historyL,rewarded_historyR,delay_start_time,goCue_start_time,reward_outcome_time,bait_left,bait_right,base_reward_probability_sum,reward_probabilityL,reward_probabilityR,reward_random_number_left,reward_random_number_right,left_valve_open_time,right_valve_open_time,block_beta,block_min,block_max,min_reward_each_block,delay_beta,delay_min,delay_max,delay_duration,ITI_beta,ITI_min,ITI_max,ITI_duration,response_duration,reward_consumption_duration,auto_waterL,auto_waterR,laser_on_trial,laser_wavelength,laser_location,laser_1_power,laser_2_power,laser_on_probability,laser_duration,laser_condition,laser_condition_probability,laser_start,laser_start_offset,laser_end,laser_end_offset,laser_protocol,laser_frequency,laser_rampingdown,laser_pulse_duration,session_wide_control,fraction_of_session,session_start_with,session_alternation,auto_train_engaged,auto_train_curriculum_name,auto_train_curriculum_version,auto_train_curriculum_schema_version,auto_train_stage,auto_train_stage_overridden,lickspout_position_x,lickspout_position_y,lickspout_position_z,reward_size_left,reward_size_right
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,4.489577e+06,4.489585e+06,2.0,False,False,4.489578e+06,4.489579e+06,4.489582e+06,False,False,0.8,0.711111,0.088889,0.963379,0.207240,0.042,0.045,20.0,2

In [9]:
# Create df_trial

nwb.df_trials = nu.create_df_trials(nwb)

nwb.df_trials.columns

Timestamps are adjusted such that `_in_session` timestamps start at the first go cue


/opt/conda/lib/python3.9/site-packages/aind_dynamic_foraging_data_utils/nwb_utils.py:457: UserWarning: Reward before choice time. This is likely due to manual rewards
  warnings.warn("Reward before choice time. This is likely due to manual rewards")


Index(['trial', 'animal_response', 'rewarded_historyL', 'rewarded_historyR',
       'bait_left', 'bait_right', 'base_reward_probability_sum',
       'reward_probabilityL', 'reward_probabilityR',
       'reward_random_number_left', 'reward_random_number_right',
       'left_valve_open_time', 'right_valve_open_time', 'block_beta',
       'block_min', 'block_max', 'min_reward_each_block', 'delay_beta',
       'delay_min', 'delay_max', 'delay_duration', 'ITI_beta', 'ITI_min',
       'ITI_max', 'ITI_duration', 'response_duration',
       'reward_consumption_duration', 'auto_waterL', 'auto_waterR',
       'laser_on_trial', 'laser_wavelength', 'laser_location', 'laser_1_power',
       'laser_2_power', 'laser_on_probability', 'laser_duration',
       'laser_condition', 'laser_condition_probability', 'laser_start',
       'laser_start_offset', 'laser_end', 'laser_end_offset', 'laser_protocol',
       'laser_frequency', 'laser_rampingdown', 'laser_pulse_duration',
       'session_wide_control', 

In [12]:
df_trial = nwb.trials.to_dataframe().copy()
df_trial.columns

Index(['start_time', 'stop_time', 'animal_response', 'rewarded_historyL',
       'rewarded_historyR', 'delay_start_time', 'goCue_start_time',
       'reward_outcome_time', 'bait_left', 'bait_right',
       'base_reward_probability_sum', 'reward_probabilityL',
       'reward_probabilityR', 'reward_random_number_left',
       'reward_random_number_right', 'left_valve_open_time',
       'right_valve_open_time', 'block_beta', 'block_min', 'block_max',
       'min_reward_each_block', 'delay_beta', 'delay_min', 'delay_max',
       'delay_duration', 'ITI_beta', 'ITI_min', 'ITI_max', 'ITI_duration',
       'response_duration', 'reward_consumption_duration', 'auto_waterL',
       'auto_waterR', 'laser_on_trial', 'laser_wavelength', 'laser_location',
       'laser_1_power', 'laser_2_power', 'laser_on_probability',
       'laser_duration', 'laser_condition', 'laser_condition_probability',
       'laser_start', 'laser_start_offset', 'laser_end', 'laser_end_offset',
       'laser_protocol', 'laser_

In [17]:
LEFT, RIGHT, IGNORE = 0, 1, 2

In [24]:
# Trial level metrics

WIN_DUR = 15
MIN_EVENTS = 2


def compute_all_trial_metrics(nwb):
    """
    Computes all trial by trial metrics

    response_rate,          fraction of trials with a response
    gocue_reward_rate,      fraction of trials with a reward
    response_reward_rate,   fraction of trials with a reward,
                            computed only on trials with a response
    choose_right_rate,      fraction of trials where chose right,
                            computed only on trials with a response

    """
    if not hasattr(nwb, "df_trials"):
        print("You need to compute df_trials: nwb_utils.create_trials_df(nwb)")
        return

    df = nwb.df_trials.copy()

#############################################################################################################################################

    df['reward'] = False  # All reward, including (collected!) autowater
    df.loc[(df.rewarded_historyL | df.rewarded_historyR
                   | df.auto_waterR | df.auto_waterL 
                    & (df.animal_response != IGNORE))  # because there may be "ignored" autowater...
                    > 0, 'reward'] = True
    df['reward_non_autowater'] = False  # Only reward from non-autowater trials (by definition, animal must not have ignored)
    df.loc[(df.rewarded_historyL | df.rewarded_historyR), 'reward_non_autowater'] = True
    
    df['non_autowater_trial'] = False
    df.loc[(df.auto_waterL==0) & (df.auto_waterR==0), 'non_autowater_trial'] = True
    df['non_autowater_finished_trial'] = df['non_autowater_trial'] & (df['animal_response'] != IGNORE)
    df['ignored_non_autowater'] = df['non_autowater_trial'] & (df['animal_response'] == IGNORE)
    df['ignored_autowater'] = ~df['non_autowater_trial'] & (df['animal_response'] == IGNORE)

    # --- Lick-related stats ---    
    all_left_licks = nwb.acquisition['left_lick_time'].timestamps[:]
    all_right_licks = nwb.acquisition['right_lick_time'].timestamps[:]
    
    # Define the start and stop time for each epoch
    lick_stats_epochs = {
        # Name: (start_time_name, stop_time_name)
        'gocue_stop': ['goCue_start_time', 'stop_time'],
        'delay_period': ['delay_start_time', 'goCue_start_time'],
        'iti': ['start_time', 'delay_start_time'],        
        }
    
    # Trial-by-trial counts
    for i in range(len(df)):
        for epoch_name, (start_time_name, stop_time_name) in lick_stats_epochs.items():
            start_time, stop_time = df.loc[i, [start_time_name, stop_time_name]]
            lick_stats = _lick_analysis_in_epoch(
                all_left_licks=all_left_licks, 
                all_right_licks=all_right_licks,
                choice=df.animal_response[i],
                start_time=start_time,
                stop_time=stop_time
            )
            
            df.loc[i, f'duration_{epoch_name}'] = stop_time - start_time
            df.loc[i, f'n_lick_left_{epoch_name}'] = lick_stats['n_lick_left']
            df.loc[i, f'n_lick_right_{epoch_name}'] = lick_stats['n_lick_right']
            df.loc[i, f'n_lick_all_{epoch_name}'] = lick_stats['n_lick_all']
            df.loc[i, f'n_lick_switches_{epoch_name}'] = lick_stats['n_lick_switches']
            df.loc[i, f'n_lick_consistency_{epoch_name}'] = lick_stats['lick_consistency']
            
            # Special treatment for gocue to stop
            if epoch_name == 'gocue_stop':
                df.loc[i, 'reaction_time'] = lick_stats['first_lick'] - df.goCue_start_time[i]
                
                # Even in ignore trials, there may be licks outside the response window, 
                # but they are invalid and should be overriden by NaN
                if df.animal_response[i] == IGNORE:
                    df.loc[i, 'reaction_time'] = np.nan
                    df.loc[i, 'n_valid_licks_left'] = 0
                    df.loc[i, 'n_valid_licks_right'] = 0
                    df.loc[i, 'n_valid_licks_all'] = 0

#############################################################################################################################################

    df["RESPONDED"] = [x in [0, 1] for x in df["animal_response"].values]
    # Rolling fraction of goCues with a response
    df["response_rate"] = (
        df["RESPONDED"].rolling(WIN_DUR, min_periods=MIN_EVENTS, center=True).mean()
    )

    # Rolling fraction of goCues with a response
    df["gocue_reward_rate"] = (
        df["earned_reward"].rolling(WIN_DUR, min_periods=MIN_EVENTS, center=True).mean()
    )

    # Rolling fraction of responses with a response
    df["RESPONSE_REWARD"] = [
        x[0] if x[1] else np.nan for x in zip(df["earned_reward"], df["RESPONDED"])
    ]
    df["response_reward_rate"] = (
        df["RESPONSE_REWARD"].rolling(WIN_DUR, min_periods=MIN_EVENTS, center=True).mean()
    )

    # Rolling fraction of choosing right
    df["WENT_RIGHT"] = [x if x in [0, 1] else np.nan for x in df["animal_response"]]
    df["choose_right_rate"] = (
        df["WENT_RIGHT"].rolling(WIN_DUR, min_periods=MIN_EVENTS, center=True).mean()
    )

    # Clean up temp columns
    drop_cols = ["RESPONDED", "RESPONSE_REWARD", "WENT_RIGHT"]
    df = df.drop(columns=drop_cols)

    return df


In [25]:
nwb.df_trials = compute_all_trial_metrics(nwb)

KeyError: "None of [Index(['goCue_start_time', 'stop_time'], dtype='object')] are in the [index]"

In [22]:
def session_metrics(nwb):
    """ 
    Compute all session level metrics

    Includes session level metadata as a temporary organizer 
    Includes majority of metrics from process_nwbs.py

    New addition: chosen_probability - average difference between the chosen probability
    and non-chosen probability / the difference between the largest and smallest probability 
    in the session
    """

    if not hasattr(nwb, 'df_trials'):
        print('You need to compute df_trials: nwb_utils.create_trials_df(nwb)')
        return
    
    df = nwb.df_trials.copy()

    # METADATA PLACEHOLDER 
    session_start_time = nwb.session_start_time
    session_date = session_start_time.strftime("%Y-%m-%d")
    subject_id_from_meta = nwb.subject.subject_id
    
    # Parse the file name for suffix
    old_re = re.match(r"(?P<subject_id>\d+)_(?P<date>\d{4}-\d{2}-\d{2})(?:_(?P<n>\d+))?\.json", 
                nwb.session_id)
    
    if old_re is not None:
        subject_id, session_date, nwb_suffix = old_re.groups()
        nwb_suffix = int(nwb_suffix) if nwb_suffix is not None else 0
    else:
        subject_id, session_date, session_json_time = re.match(r"(?P<subject_id>\d+)_(?P<date>\d{4}-\d{2}-\d{2})(?:_(?P<time>.*))\.json", 
                            nwb.session_id).groups()
        nwb_suffix = int(session_json_time.replace('-', ''))
    
    # Verify metadata matches
    assert subject_id == subject_id_from_meta, f"Subject name from the metadata ({subject_id_from_meta}) does not match "\
                                               f"that from json name ({subject_id})!!"
    assert session_date == session_date, f"Session date from the metadata ({session_date}) does not match "\
                                                   f"that from json name ({session_date})!!"
    
    # Create session index
    session_index = pd.MultiIndex.from_tuples([(subject_id, session_date, nwb_suffix)], 
                                            names=['subject_id', 'session_date', 'nwb_suffix'])

    # Get metadata from nwb.scratch
    meta_dict = nwb.scratch['metadata'].to_dataframe().iloc[0].to_dict()
    
    # Calculate performance metrics
    n_total_trials = len(df)
    n_finished_trials = (df.animal_response != IGNORE).sum()
    
    # Actual foraging trials (autowater excluded)
    n_total_trials_non_autowater = df.non_autowater_trial.sum()
    n_finished_trials_non_autowater = df.non_autowater_finished_trial.sum()
    
    n_reward_trials_non_autowater = df.reward_non_autowater.sum()
    reward_rate_non_autowater_finished = n_reward_trials_non_autowater / n_finished_trials_non_autowater if n_finished_trials_non_autowater > 0 else np.nan
    
    # Calculate foraging efficiency
    foraging_eff, foraging_eff_random_seed = compute_foraging_efficiency(
        baited='without bait' not in nwb.protocol.lower(),
        choice_history=df.animal_response.map({0: 0, 1: 1, 2: np.nan}).values,
        reward_history=df.rewarded_historyL | df.rewarded_historyR,
        p_reward=[
            df.reward_probabilityL.values,
            df.reward_probabilityR.values,
        ],
        random_number=[
            df.reward_random_number_left.values,
            df.reward_random_number_right.values,
        ],
        autowater_offered=(df.auto_waterL == 1) | (df.auto_waterR == 1),
    )
    
    # Override foraging_eff_random_seed for old bpod sessions
    if 'bpod' in nwb.session_description:
        foraging_eff_random_seed = nwb.get_scratch('metadata')['foraging_efficiency_with_actual_random_seed'].values[0]
    
    finished_rate = n_finished_trials_non_autowater / n_total_trials_non_autowater if n_total_trials_non_autowater > 0 else np.nan
    
    # New Metrics

    # Probability chosen calculation 
    probability_chosen = []
    probability_not_chosen = []

    for _, row in df.iterrows():
        if row.animal_response == 2:
            probability_chosen.append(np.nan)
            probability_not_chosen.append(np.nan)
        elif row.animal_response == 0:  # Chosen = left choice left probability, not chosen = left choice right probability 
            probability_chosen.append(row.reward_probabilityL)
            probability_not_chosen.append(row.reward_probabilityR)
        else: # Chosen = right choice right probability, not chosen = right choice left probability 
            probability_chosen.append(row.reward_probabilityR)
            probability_not_chosen.append(row.reward_probabilityL)

    df['probability_chosen'] = probability_chosen
    df['probability_not_chosen'] = probability_not_chosen

    # Calculate the chosen probability
    average = (df['probability_chosen'] - df['probability_not_chosen'])
    
    p_larger_global = max(df['probability_chosen'].max(), df['probability_not_chosen'].max())
    
    p_smaller_global = min(df['probability_chosen'].min(), df['probability_not_chosen'].min())
    
    mean_difference = average.mean()
    chosen_probability = mean_difference / (p_larger_global - p_smaller_global)


    # Pack all data
    dict_meta = {
        # Basic metadata
        'rig': meta_dict['box'],        
        'user_name': nwb.experimenter[0],
        'task': nwb.protocol,
        
        # New metric
        'chosen_probability': chosen_probability,

        # Trial counts and rates
        'total_trials': n_total_trials_non_autowater,
        'finished_trials': n_finished_trials_non_autowater,
        'finished_rate': finished_rate,
        'total_trials_with_autowater': n_total_trials,
        'finished_trials_with_autowater': n_finished_trials,
        'finished_rate_with_autowater': n_finished_trials / n_total_trials,
        
        # Reward and foraging metrics
        'reward_trials': n_reward_trials_non_autowater,
        'reward_rate': reward_rate_non_autowater_finished,
        'foraging_eff': foraging_eff,
        'foraging_eff_random_seed': foraging_eff_random_seed,
        'foraging_performance': foraging_eff * finished_rate,
        'foraging_performance_random_seed': foraging_eff_random_seed * finished_rate,
        
        # Timing metrics
        'reaction_time_median': df.loc[:, 'reaction_time'].median(),
        'reaction_time_mean': df.loc[:, 'reaction_time'].mean(),
        'early_lick_rate': (df.loc[:, 'n_lick_all_delay_period'] > 0).sum() / n_total_trials,
        
        # Double dipping metrics
        'double_dipping_rate_finished_trials': 
            (df.loc[(df.animal_response != IGNORE), 'n_lick_switches_gocue_stop'] > 0).sum() 
            / (df.animal_response != IGNORE).sum(),
        'double_dipping_rate_finished_reward_trials':
            (df.loc[df.reward, 'n_lick_switches_gocue_stop'] > 0).sum()  
            / df.reward.sum(),
        'double_dipping_rate_finished_noreward_trials':
            (df.loc[(df.animal_response != IGNORE) & (~df.reward), 'n_lick_switches_gocue_stop'] > 0).sum() 
            / ((df.animal_response != IGNORE) & (~df.reward)).sum(),
            
        # Lick consistency metrics
        'lick_consistency_mean_finished_trials': 
            df.loc[(df.animal_response != IGNORE), 'n_lick_consistency_gocue_stop'].mean(),
        'lick_consistency_mean_finished_reward_trials': 
            df.loc[df.reward, 'n_lick_consistency_gocue_stop'].mean(),
        'lick_consistency_mean_finished_noreward_trials': 
            df.loc[(df.animal_response != IGNORE) & (~df.reward), 'n_lick_consistency_gocue_stop'].mean(),
    }
    
    # Create DataFrame with hierarchical columns
    df_meta = pd.DataFrame(dict_meta, 
                          index=session_index)
    df_meta.columns = pd.MultiIndex.from_product([['metadata'], dict_meta.keys()],
                                                names=['type', 'variable'])
    
    return df_meta

In [23]:
final_df = session_metrics(nwb)

KeyError: 'reaction_time'